In [20]:
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import json
import mysql.connector

On prend l'url de la page souhaité et on récupére tout son HTML.
Ici je récupère la première apparation du tag <h3> grace a find. 
Mais aussi tout les tag <span> qui se trouve dans se HTML grace a find_all.

In [57]:
URL = "https://www.mon-horoscope-du-jour.com/horoscopes/quotidien/poissons.htm"
page = requests.get(URL)
soup = bs(page.content, "html.parser")
h3 = soup.find('h3')
span = soup.find_all('span',class_="horo_content_stof")
print(span)
print(h3.prettify())

[<span class="horo_content_stof"></span>, <span class="horo_content_stof">★★</span>, <span class="horo_content_stof">★</span>, <span class="horo_content_stof"></span>, <span class="horo_content_stof"></span>]
<h3>
 Humeur
</h3>



In [48]:
headers = {
'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.79 Safari/537.36'
}

url = 'https://www.horoscope.fr/horoscopes/aujourdhui/poissons'

r = requests.get(url, headers=headers)
soup = bs(r.text, 'html.parser')
data = json.loads(soup.select_one('script[id="__NEXT_DATA__"]').text)
magic_stuffs = data['props']['pageProps']['initialProps']['horoscope']['overviews']
other_stuffs = data['props']['pageProps']['initialProps']['horoscope']['contents']
df = pd.json_normalize(magic_stuffs)
dd = pd.json_normalize(other_stuffs)
print(df)
print(dd['sections'])


   rating        title                                            iconUrl  \
0       5        AMOUR  https://cdn.tlmq.fr/mbe/horoscope/rating_5_v1.png   
1       3      TRAVAIL  https://cdn.tlmq.fr/mbe/horoscope/rating_3_v1.png   
2       5    BIEN-ÊTRE  https://cdn.tlmq.fr/mbe/horoscope/rating_5_v1.png   
3       5  VIE SOCIALE  https://cdn.tlmq.fr/mbe/horoscope/rating_5_v1.png   
4       4     AMBIANCE  https://cdn.tlmq.fr/mbe/horoscope/rating_4_v1.png   
5       4     FINANCES  https://cdn.tlmq.fr/mbe/horoscope/rating_4_v1.png   

  titleColor     anchorId  
0       #000         love  
1       #000       career  
2       #000    wellbeing  
3       #000  social_life  
4       #000         mood  
5       #000     finances  
0    [{'id': 'mood.astro_mood', 'title': 'Votre act...
1    [{'id': 'love.in_relationship', 'title': 'Si v...
2    [{'id': 'career.work', 'title': 'Votre travail...
3    [{'id': 'wellbeing.wellbeing', 'title': 'Votre...
4    [{'id': 'social_life.family', 'title': 

In [214]:
for paragraphe in paragraphes:
    paragraphe = paragraphe.find('p')
    print('•' +  paragraphe.text)

i = 0
for truc in trucs:
    print(titles[i].text)
    i = i +1
    if truc.text == '':        
        print("5/5")
    elif truc.text == "★":
        print("4/5")
    elif truc.text == "★★":
        print("3/5")
    elif truc.text == "★★★":
        print("2/5")
    elif truc.text == "★★★★":
        print("1/5")
    else:
        print("0/5")
        

•Vous devez sérieusement veiller à vos dépenses et ne pas vous laisser tenter exagérément par le luxe et la volupté. Tous les excès se paient un jour, parfois sur la balance de la salle de bains, parfois sur les relevés de compte...
•Vous devez prendre d'importantes décisions dans vos relations sentimentales ne doivent pas vous empêcher d'avancer. Seules vous conviennent les relations qui s'harmonisent avec votre avenir et qui ne vous obligent pas à laisser passer les opportunités.
•Il vous faudra aujourd'hui encore poser vos limites avec fermeté pour ne pas vous faire ponctionner. De plus, il faudra surmonter une opposition pour chasser un malentendu. Vous pourrez éclaircir les choses dans la bonne direction, avec une bonne dose de calme.
•Si l'on vous propose un temps de formation ou d'enrichir votre fiche de poste, n'hésitez pas. L'offre peut être pertinente, vous aurez ainsi l'occasion de développer vos connaissances. Il n'est pas négligeable, d'ajouter une corde à son arc.
•Vous n

In [101]:
reqText1 = soup.find('p').text
print(reqText1)


La Lune en Taureau et Vénus en Scorpion vous font chercher la perle rare ! La réussite de votre vie affective compte. Écoutez votre intuition pour choisir.


In [46]:
cnx = mysql.connector.connect(user='suki', password='537164',
                              host='127.0.0.1',
                              database='magic8ball')
print ("Connected")

cursor = cnx.cursor()

add_horoscope = ("INSERT INTO horoscopes" +
               "(site, type, title, text, rating,created_at) "
               "VALUES (%s,%s,%s,%s,%s,%s)")

id = int(1)
site = url
type = str(df['title'][1])
title =  str(df['title'][1])
text =  str(dd['sections'][1][1]['text'])
rating =  str(df['rating'][0])
created_at = datetime.now()

val = (site,type,title,text,rating,created_at)
print(val)
# Insert new employee
cursor.execute(add_horoscope,val)
cnx.commit()

Connected
('https://www.horoscope.fr/horoscopes/aujourdhui/poissons', 'TRAVAIL', 'TRAVAIL', "Le célibat peut cesser rapidement. Un coup de foudre est tout à fait possible. Il peut venir d'une attraction physique. Une personne avec un puissant magnétisme peut faire tourner votre tête. Écoutez vos pulsions. Ce désir peut vous entraîner vers une aventure. Vous pouvez faire le premier pas pour provoquer le destin\xa0! De belles émotions vous attendent.", '5', datetime.datetime(2022, 11, 10, 13, 8, 13, 82316))
